In [6]:
!pip install transformers datasets torch
!pip install accelerate -U

Some weights of the model checkpoint at google/muril-base-cased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [9]:
from transformers import AutoTokenizer, AutoModelForMaskedLM

# Load the MuRIL tokenizer and model
model_name = "google/muril-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForMaskedLM.from_pretrained(model_name)


/opt/homebrew/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of the model checkpoint at google/muril-base-cased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializi

In [10]:
from datasets import load_dataset

# Load your Hindi text file into a dataset
dataset = load_dataset('text', data_files={'train': 'purana1.txt'})

# Preview a few samples from the dataset
print(dataset['train'][0]['text'])

4. SHIVA MAHAPURANA 


In [11]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples['text'], truncation=True, padding='max_length')
    tokenized_inputs['labels'] = tokenized_inputs['input_ids'].copy()  # Copy input_ids to labels
    return tokenized_inputs

def tokenize_function(examples):
    return tokenizer(examples['text'], truncation=True, padding='max_length')

# Tokenize the dataset
tokenized_datasets = dataset.map(tokenize_function, batched=True)

In [13]:
from transformers import DataCollatorForLanguageModeling

# Data collator for masking tokens
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, 
    mlm=True,  # Enable masked language modeling
    mlm_probability=0.15  # 15% of tokens will be masked
)

In [ ]:
from transformers import Trainer, TrainingArguments

# Define training arguments
# training_args = TrainingArguments(
#     output_dir='./results',          # Output directory
#     num_train_epochs=1,              # Number of training epochs
#     per_device_train_batch_size=4,   # Batch size per device during training
#     save_steps=1000,               # Save checkpoint every 10k steps
#     save_total_limit=2,              # Keep only 2 checkpoints
#     logging_dir='./logs',            # Directory for logs
# )

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',          # Output directory
    num_train_epochs=2,              # Number of training epochs
    per_device_train_batch_size=4,   # Batch size per device during training
    save_steps=1000,                 # Save checkpoint every 10k steps
    save_total_limit=2,              # Keep only 2 checkpoints
    logging_dir='./logs',            # Directory for logs
    fp16=False,                      # Disable mixed-precision training (for GPU)
    no_cuda=True                     # Ensure CUDA is not attempted on M1/M3 Mac
)



In [ ]:

# Set up the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    data_collator=data_collator  # Add the collator for masking
)


In [34]:
# Train the model
trainer.train()

 34%|███▍      | 500/1471 [55:33<1:32:31,  5.72s/it]  

{'loss': 3.166, 'grad_norm': 123.60968017578125, 'learning_rate': 3.3004758667573084e-05, 'epoch': 0.34}


 68%|██████▊   | 1000/1471 [1:50:44<46:01,  5.86s/it]  

{'loss': 2.795, 'grad_norm': 162.5903778076172, 'learning_rate': 1.600951733514616e-05, 'epoch': 0.68}


ValueError: You are trying to save a non contiguous tensor: `bert.encoder.layer.0.attention.self.query.weight` which is not allowed. It either means you are trying to save tensors which are reference of each other in which case it's recommended to save only the full tensors, and reslice at load time, or simply call `.contiguous()` on your tensor to pack it before saving.

In [35]:
# Ensure all tensors are contiguous before saving
for param in model.parameters():
    param.data = param.data.contiguous()

# Now save the model and tokenizer
model.save_pretrained('./fine_tuned_muril')
tokenizer.save_pretrained('./fine_tuned_muril')


('./fine_tuned_muril/tokenizer_config.json',
 './fine_tuned_muril/special_tokens_map.json',
 './fine_tuned_muril/vocab.txt',
 './fine_tuned_muril/added_tokens.json',
 './fine_tuned_muril/tokenizer.json')